In [1]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-02-18_19:11
E:\RIS\Staff Folders\Samuel\Requests\SH\Forms_Request


In [2]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')
sdb = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG')
sqldb = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')


Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-02-18 19:12:03

Additional database connection details required:
User name (sip):soge
Password (sip)········
Database connection (PG) to sip on dotpgsql01 - user: soge 
Connection established 2020-02-18 19:12:06
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-02-18 19:12:07
Query run in 1000 microseconds
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [3]:
sdb.connect()

Database connection (PG) to sip on dotpgsql01 - user: soge 
Connection established 2020-02-18 19:12:09


### All nodeids on Corridor Stretches of SIPs 1407 and 1429

In [4]:
sip_nodes = sdb.dfquery("""
                
WITH data AS(
SELECT sp.pid, spg.segmentid, cl.mft, masteridfr, masteridto
FROM public.sip_projects sp
JOIN public.sip_projects_geo spg
ON sp.pid=spg.pid_fk
JOIN clion cl
ON spg.segmentid = cl.segmentid::int
WHERE sp.pid in (1407, 1429)
)

SELECT DISTINCT nodeid 
FROM clion_node 
WHERE masterid in (SELECT DISTINCT masteridfr masterid
                   FROM data

                   UNION

                   SELECT DISTINCT masteridto masterid
                   FROM data)
""")


In [5]:
nodes = tuple(map(int,(sip_nodes.nodeid)))

### All NYPD Crash information related to SIPs 1407 and 1429 from 1/1/2018 to Present

In [6]:
data = sqldb.dfquery("""

SELECT DISTINCT coalesce(core.nodeid, core.lion_node_number) NODEID
      ,core.INTEGRATION_ID
      ,core.ACCIDENT_DT 
      ,left(stuff(STUFF(right('000000'+cast(core.ACCIDENT_TIME_WID AS VARCHAR),6),5,0,':'),3,0,':'), 5) ACCIDENT_TIME
      ,core.INJURED_CNT
      ,core.TRAFFIC_CONTROL
      ,core.ACCIDENT_DIAGRAM
      ,vic.PED_NONPED
      ,vic.PED_ACTION
FROM [FORMS].[dbo].[WC_ACCIDENT_F] core
LEFT JOIN (SELECT DISTINCT ACCIDENT_ID, PED_ACTION, PED_NONPED, INJ_KILLED
           FROM [FORMS].[dbo].[WC_ACCIDENT_VICTIM_F]
           WHERE INJ_KILLED = 'Injured') vic
ON core.integration_id = vic.accident_id
WHERE YEAR(core.ACCIDENT_DT)>2017
AND coalesce(core.VOID_STATUS_CD , 'N') ='N'
AND core.nodeid in {}
AND coalesce(core.nodeid, core.lion_node_number) is not null


""".format(nodes)
)

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [7]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-02-18 19:12:22


In [8]:
st_names = cdb.dfquery("""  

SELECT st_names.nodeid::int "NODEID", array_agg(street) "STREET NAMES", ST_X(geom) "X",  ST_Y(geom) "Y"
FROM (SELECT distinct nodeidfrom nodeid, street, featuretyp
      FROM lion 
      WHERE nodeidfrom::int in {n}
      UNION

      SELECT DISTINCT nodeidto nodeid, street, featuretyp
      FROM lion 
      WHERE nodeidto::int in {n}
      ) st_names
LEFT JOIN node n
ON n.nodeid::int = st_names.nodeid::int
WHERE featuretyp in ('0', '6', 'C')
GROUP BY st_names.NODEID,  ST_X(geom),  ST_Y(geom)""".format(n=nodes)
                      
)


In [9]:
data = data.merge(st_names, on='NODEID', how = 'left')

In [10]:
data.to_excel('forms_request.xlsx',index=False)